# Queries Notebook


* **Author**: Roberto Rodriguez (@Cyb3rWard0g)
* **Project**: Infosec Jupyter Book
* **Public Organization**: [Open Threat Research](https://github.com/OTRF)
* **License**: [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)
    

## Import Libraries

In [1]:
from py2neo import Graph

## Initialize Graph Variable

In [2]:
graph = Graph(password='wardog')

## Community Cypher Queries

## Servers a user can RDP into
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find servers a user can RDP into.


In [3]:
graph.run(
'''
MATCH p=(g:Group)-[:CanRDP]->(c:Computer)
WHERE  g.objectid ENDS WITH '-513' AND c.operatingsystem CONTAINS 'Server'
RETURN p   
'''
).data()

[]

## View all GPOs that contain a keyword
**Author:** Ryan Hausknecht (@haus3c)

**Description:** View all GPOs that contain a keyword


In [4]:
graph.run(
'''
MATCH (n:GPO)
WHERE n.name CONTAINS "DOMAIN"
RETURN n
'''
).data()

[{'n': (_10:Base:GPO {distinguishedname: 'CN={6AC1786C-016F-11D2-945F-00C04fB984F9},CN=Policies,CN=System,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', gpcpath: '\\\\contoso.local\\sysvol\\contoso.local\\Policies\\{6AC1786C-016F-11D2-945F-00C04fB984F9}', highvalue: false, name: 'DEFAULT DOMAIN CONTROLLERS POLICY@CONTOSO.LOCAL', objectid: '6930F38E-81B5-497F-B18B-E770DF862D89'})},
 {'n': (_120:Base:GPO {distinguishedname: 'CN={31B2F340-016D-11D2-945F-00C04FB984F9},CN=Policies,CN=System,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', gpcpath: '\\\\contoso.local\\sysvol\\contoso.local\\Policies\\{31B2F340-016D-11D2-945F-00C04FB984F9}', highvalue: false, name: 'DEFAULT DOMAIN POLICY@CONTOSO.LOCAL', objectid: '5E11EF13-E6EE-4600-9EF7-BD5220CCE469'})}]

## Domain Users Groups with Interesting ACEs
**Author:** Ryan Hausknecht (@haus3c), Roberto Rodriguez (@Cyb3rWard0g)

**Description:** Find interesting privileges/ACEs that have been configured to DOMAIN USERS group


In [5]:
graph.run(
'''
MATCH (m:Group)
WHERE m.name =~ 'DOMAIN USERS@CONTOSO.LOCAL'
MATCH p=(m)-[r:Owns|WriteDacl|GenericAll|WriteOwner|ExecuteDCOM|GenericWrite|AllowedToDelegate|ForceChangePassword]->(n:Computer)
RETURN p
'''
).data()

[]

## Top 10 Computers with Most Admins
**Author:** Walter.Legowski (@SadProcessor)

**Description:** List of top 10 computers with most admins


In [6]:
graph.run(
'''
MATCH (n:User),(m:Computer),(n)-[r:AdminTo]->(m)
WHERE NOT n.name STARTS WITH 'ANONYMOUS LOGON' AND NOT n.name='' WITH m,count(r) as rel_count 
ORDER BY rel_count desc 
LIMIT 10 
MATCH (m)<-[r:AdminTo]-(n) 
RETURN n,r,m
'''
).data()

[{'n': (_11:Base:User {admincount: true, description: 'Built-in account for administering the computer/domain', distinguishedname: 'CN=Administrator,CN=Users,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', dontreqpreauth: false, enabled: true, hasspn: false, highvalue: false, lastlogon: 1586793173.0, lastlogontimestamp: 1586738080.0, name: 'ADMINISTRATOR@CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-500', owned: false, passwordnotreqd: false, pwdlastset: 1560189819.0, pwdneverexpires: true, sensitive: false, serviceprincipalnames: [], sidhistory: [], unconstraineddelegation: false}),
  'r': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL),
  'm': (_41:Base:Computer {distinguishedname: 'CN=DC-2016-001,OU=Domain Controllers,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', enabled: true, haslaps: false, highvalue: false, lastlogontimestamp: 1586550660.0, name: 'DC-2016-001.CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-17

## Map Domain Trusts
**Author:** Walter.Legowski (@SadProcessor)

**Description:** Map domain trusts


In [7]:
graph.run(
'''
MATCH (n:Domain) MATCH p=(n)-[r]-() RETURN p
'''
).data()

[{'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(RESEARCH AND DEVELOPMENT@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(ACCOUNTING@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)<-[:GpLink {enforced: false, isacl: false}]-(DEFAULT DOMAIN POLICY@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(CONTOSO USERS@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(SQLUSERS@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(DOMAIN CONTROLLERS@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(PCI ENCLAVE@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(OU-CONTROL@CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(WIN-2016-001.CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(DESKTOP-4AMBQF0.CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(WIN10-001.CONTOSO.LOCAL)},
 {'p': (CONTOSO.LOCAL)-[:Contains {isacl: false}]->(KRBTGT@CONTO

## High Value Target Group
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Show all high value target group


In [8]:
graph.run(
'''
MATCH p=(n:User)-[r:MemberOf*1..]->(m:Group {highvalue:true})
RETURN p
'''
).data()

[{'p': (ADMINISTRATOR@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ENTERPRISE ADMINS@CONTOSO.LOCAL)},
 {'p': (ACHILES@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ENTERPRISE ADMINS@CONTOSO.LOCAL)},
 {'p': (SQL01@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (DAVID.MCGUIRE@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (ADMINISTRATOR@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (JPRAGER@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)},
 {'p': (SQL2016@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)},
 {'p': (DAVID.MCGUIRE@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)},
 {'p': (BOB.ACCOUNTING@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(D

## Shortest Path to DA Groups from Domain Users Groups
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Shortest paths to Domain Admins group from the Domain Users group


In [9]:
graph.run(
'''
MATCH (g:Group)
WHERE g.name =~ 'DOMAIN USERS@.*'
MATCH (g1:Group)
WHERE g1.name =~ 'DOMAIN ADMINS@.*'
OPTIONAL MATCH p=shortestPath((g)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct|SQLAdmin*1..]->(g1))
RETURN p
'''
).data()

[{'p': (DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)}]

## ASP-REQ Roastable Users
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find user that doesn’t require kerberos pre-authentication (aka AS-REP Roasting)


In [10]:
graph.run(
'''
MATCH (u:User {dontreqpreauth: true})
RETURN u
'''
).data()

[]

## Unprivileged Users with Rights to Add Members to Groups
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find if unprivileged users have rights to add members into groups


In [11]:
graph.run(
'''
MATCH (n:User {admincount:False})
MATCH p=allShortestPaths((n)-[r:AddMember*1..]->(m:Group))
RETURN p
'''
).data()

[]

## Users that Logged in ithin Threshold
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find users that logged in within the last 90 days. Change 90 to whatever threshold you want.


In [12]:
graph.run(
'''
MATCH (u:User)
WHERE u.lastlogon < (datetime().epochseconds - (90 * 86400)) and NOT u.lastlogon IN [-1.0, 0.0]
RETURN u.name
'''
).data()

[{'u.name': 'JEFFMCJUNKIN@CONTOSO.LOCAL'},
 {'u.name': 'SQL2014@CONTOSO.LOCAL'},
 {'u.name': 'JBUI@CONTOSO.LOCAL'},
 {'u.name': 'BOB.ACCOUNTING@CONTOSO.LOCAL'},
 {'u.name': 'DPOLOJAC@CONTOSO.LOCAL'},
 {'u.name': 'DHOHNSTEIN@CONTOSO.LOCAL'},
 {'u.name': 'LPAINE@CONTOSO.LOCAL'},
 {'u.name': 'SQL2016@CONTOSO.LOCAL'},
 {'u.name': 'SQL2017@CONTOSO.LOCAL'},
 {'u.name': 'SQL01@CONTOSO.LOCAL'}]

## Shortest Path to DA Groups from Non-Privileged Domain Users
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Shortest paths to Domain Admins group from non privileged users (AdminCount=false)


In [13]:
graph.run(
'''
MATCH (n:User {admincount:false}),(m:Group {name:'DOMAIN ADMINS@CONTOSO.LOCAL'}),p=shortestPath((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct*1..]->(m))
RETURN p
'''
).data()

[]

## Unsupported OSs
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find unsupported OSs


In [14]:
graph.run(
'''
MATCH (H:Computer) WHERE H.operatingsystem =~ '.*(2000|2003|2008|xp|vista|7|me)*.'
RETURN H.name
'''
).data()

[{'H.name': 'PCI-SERVER-001.CONTOSO.LOCAL'},
 {'H.name': 'DC-2016-001.CONTOSO.LOCAL'},
 {'H.name': 'WIN-2016-001.CONTOSO.LOCAL'},
 {'H.name': 'DESKTOP-4AMBQF0.CONTOSO.LOCAL'},
 {'H.name': 'WIN10-001.CONTOSO.LOCAL'}]

## Top 10 Users with Most Sessions
**Author:** Walter.Legowski (@SadProcessor)

**Description:** List Top 10 Users with Most Sessions


In [15]:
graph.run(
'''
MATCH (n:User),(m:Computer),(n)<-[r:HasSession]-(m) 
WHERE NOT n.name STARTS WITH 'ANONYMOUS LOGON' 
AND NOT n.name='' WITH n, 
count(r) as rel_count 
order by rel_count desc 
LIMIT 10 
MATCH (m)-[r:HasSession]->(n) 
RETURN n,r,m
'''
).data()

[{'n': (_165:Base:User {admincount: true, displayname: 'Andrew Chiles', distinguishedname: 'CN=Andrew Chiles,OU=PCI Admins,OU=PCI Enclave,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', dontreqpreauth: false, enabled: true, hasspn: false, highvalue: false, lastlogon: -1.0, lastlogontimestamp: -1.0, name: 'ACHILES@CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-125934', owned: false, passwordnotreqd: false, pwdlastset: 1586797965.0, pwdneverexpires: true, sensitive: false, serviceprincipalnames: [], sidhistory: [], unconstraineddelegation: false}),
  'r': (RD-WIN10-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ACHILES@CONTOSO.LOCAL),
  'm': (_175:Base:Computer {distinguishedname: 'CN=RD-WIN10-001,OU=R&D Computers,OU=Research and Development,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', enabled: true, haslaps: false, highvalue: false, lastlogontimestamp: -1.0, name: 'RD-WIN10-001.CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-125944', owned: false

## Users with Passwords Last Set withing Threshold
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find users with passwords last set thin the last 90 days. Change 90 to whatever threshold you want.


In [16]:
graph.run(
'''
MATCH (u:User)
WHERE u.lastlogon < (datetime().epochseconds - (90 * 86400)) and NOT u.lastlogon IN [-1.0, 0.0]
RETURN u.name
'''
).data()

[{'u.name': 'JEFFMCJUNKIN@CONTOSO.LOCAL'},
 {'u.name': 'SQL2014@CONTOSO.LOCAL'},
 {'u.name': 'JBUI@CONTOSO.LOCAL'},
 {'u.name': 'BOB.ACCOUNTING@CONTOSO.LOCAL'},
 {'u.name': 'DPOLOJAC@CONTOSO.LOCAL'},
 {'u.name': 'DHOHNSTEIN@CONTOSO.LOCAL'},
 {'u.name': 'LPAINE@CONTOSO.LOCAL'},
 {'u.name': 'SQL2016@CONTOSO.LOCAL'},
 {'u.name': 'SQL2017@CONTOSO.LOCAL'},
 {'u.name': 'SQL01@CONTOSO.LOCAL'}]

## Shortest Path to DA Groups from Computers
**Author:** Ryan Hausknecht (@haus3c), Roberto Rodriguez (@Cyb3rWard0g)

**Description:** Shortest paths to Domain Admins group from computers


In [17]:
graph.run(
'''
MATCH (n:Computer),(m:Group {name:'DOMAIN ADMINS@CONTOSO.LOCAL'}),p=shortestPath((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct*1..]->(m))
RETURN p
'''
).data()

[{'p': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (RD-WIN10-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ACHILES@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ENTERPRISE ADMINS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)}]

## All Domain Users CanRDP Edges Against all Computers
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find only the CanRDP privileges (edges) of the domain users against the domain computers


In [18]:
graph.run(
'''
MATCH p1=shortestPath(((u1:User)-[r1:MemberOf*1..]->(g1:Group)))
MATCH p2=(u1)-[:CanRDP*1..]->(c:Computer)
RETURN p2
'''
).data()

[]

## All Domain Users AdminTo Edges Against all Computers
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find only the AdminTo privileges (edges) of the domain users against the domain computers


In [19]:
graph.run(
'''
MATCH p1=shortestPath(((u1:User)-[r1:MemberOf*1..]->(g1:Group)))
MATCH p2=(u1)-[:AdminTo*1..]->(c:Computer)
RETURN p2
'''
).data()

[{'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 

## Active Users Sessions in all Domain Computers
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find the active user sessions on all domain computers


In [20]:
graph.run(
'''
MATCH p1=shortestPath(((u1:User)-[r1:MemberOf*1..]->(g1:Group)))
MATCH p2=(c:Computer)-[*1]->(u1)
RETURN p2
'''
).data()

[{'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p2': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO

## Kerberoastable Users with a path to DA
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find Kerberoastable Users with a path to DA


In [21]:
graph.run(
'''
MATCH (u:User {hasspn:true})
MATCH (g:Group)
WHERE g.name CONTAINS 'DOMAIN ADMINS' MATCH p = shortestPath( (u)-[*1..]->(g) )
RETURN p
'''
).data()

[{'p': (KRBTGT@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (SQL2014@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)}]

## Kerberoastable Users
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find All Users with an SPN/Find all Kerberoastable Users


In [22]:
graph.run(
'''
MATCH (n:User)WHERE n.hasspn=true
RETURN n.name
'''
).data()

[{'n.name': 'KRBTGT@CONTOSO.LOCAL'}, {'n.name': 'SQL2014@CONTOSO.LOCAL'}]

## Specific Users Edges to All Nodes
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find all Edges that a specific user has against all the nodes (HasSession is not calculated, as it is an edge that comes from computer to user, not from user to computer)


In [23]:
graph.run(
'''
MATCH (n:User)
WHERE n.name =~ 'JEFFMCJUNKIN@CONTOSO.LOCAL'
MATCH (m)
WHERE NOT m.name = n.name
MATCH p=allShortestPaths((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct|SQLAdmin*1..]->(m))
RETURN p
LIMIT 10
'''
).data()

[{'p': (JEFFMCJUNKIN@CONTOSO.LOCAL)-[:GenericAll {isacl: true, isinherited: false}]->(CONTOSO USERS@CONTOSO.LOCAL)},
 {'p': (JEFFMCJUNKIN@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: true}]->(SQLUSERS@CONTOSO.LOCAL)},
 {'p': (JEFFMCJUNKIN@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteOwner {isacl: true, isinherited: true}]->(SQLUSERS@CONTOSO.LOCAL)},
 {'p': (JEFFMCJUNKIN@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: true}]->(PCI ENCLAVE@CONTOSO.LOCAL)},
 {'p': (JEFFMCJUNKIN@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteOwner {isacl: true, isinherited: true}]->(

## Shortest Path to DA Groups from Domain Groups
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Shortest paths to Domain Admins group from all domain groups


In [24]:
graph.run(
'''
MATCH (n:Group)
WHERE NOT n.name = 'DOMAIN ADMINS@CONTOSO.LOCAL'
MATCH (m:Group {name:'DOMAIN ADMINS@CONTOSO.LOCAL'}),p=shortestPath((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct*1..]->(m))
RETURN p
'''
).data()

[{'p': (ENTERPRISE ADMINS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (ACCOUNT OPERATORS@CONTOSO.LOCAL)-[:GenericAll {isacl: true, isinherited: false}]->(PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (TERMINAL SERVER LICENSE SERVERS@CONTOSO.LOCAL)-[:GenericAll {isacl: true, isinherited: true}]->(SVC_SHARPHOUND@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)}]

## Shortest Path to DA Groups from Non-Privileged Domain Groups
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Shortest paths to Domain Admins group from non-privileged groups (AdminCount=false)


In [25]:
graph.run(
'''
MATCH (n:Group {admincount:false}),(m:Group {name:'DOMAIN ADMINS@CONTOSO.LOCAL'}),p=shortestPath((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct*1..]->(m))
RETURN p
'''
).data()

[{'p': (TERMINAL SERVER LICENSE SERVERS@CONTOSO.LOCAL)-[:GenericAll {isacl: true, isinherited: true}]->(SVC_SHARPHOUND@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ADMINISTRATORS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)}]

## Shortest Path to DA Groups from Computers Excluding DCs
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Shortest paths to Domain Admins group from computers excluding potential DCs (based on ldap/ and GC/ spns)


In [26]:
graph.run(
'''
WITH '(?i)ldap/.*' as regex_one WITH '(?i)gc/.*' as regex_two
MATCH (n:Computer)
WHERE NOT ANY(item IN n.serviceprincipalnames WHERE item =~ regex_two OR item =~ regex_two )
MATCH(m:Group {name:"DOMAIN ADMINS@CONTOSO.LOCAL"}),p=shortestPath((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct*1..]->(m))
RETURN p
'''
).data()

[{'p': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)},
 {'p': (RD-WIN10-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ACHILES@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(ENTERPRISE ADMINS@CONTOSO.LOCAL)-[:WriteDacl {isacl: true, isinherited: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)}]

## Unprivileged Users Edges to All Nodes
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find all the Edges that any UNPRIVILEGED user (based on the admincount:False) has against all the nodes


In [27]:
graph.run(
'''
MATCH (n:User {admincount:False})
MATCH (m)
WHERE NOT m.name = n.name
MATCH p=allShortestPaths((n)-[r:MemberOf|HasSession|AdminTo|AllExtendedRights|AddMember|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|CanRDP|ExecuteDCOM|AllowedToDelegate|ReadLAPSPassword|Contains|GpLink|AddAllowedToAct|AllowedToAct|SQLAdmin*1..]->(m))
RETURN p
LIMIT 10
'''
).data()

[{'p': (GUEST@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(GUESTS@CONTOSO.LOCAL)},
 {'p': (GUEST@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN GUESTS@CONTOSO.LOCAL)},
 {'p': (SQL_HQ_PRIMARY@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN COMPUTERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(AUTHENTICATED USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(USERS@CONTOSO.LOCAL)},
 {'p': (SQL_HQ_PRIMARY@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN COMPUTERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(AUTHENTICATED USERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(PRE-WINDOWS 2000 COMPATIBLE ACCESS@CONTOSO.LOCAL)},
 {'p': (SQL_HQ_PRIMARY@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN COMPUTERS@CONTOSO.LOCAL)},
 {'p': (SQL_HQ_PRIMARY@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN COMPUTERS@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(EVERYONE@CONTOSO.LOCAL)},
 {'p': (SQL_HQ_PRIMARY@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN COMPUTERS@CONTOSO.LOCAL)-[:MemberOf {isac

## Unprivileged Users ACL abusing other Users
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find interesting edges related to “ACL Abuse” that uprivileged users have against other users


In [28]:
graph.run(
'''
MATCH (n:User {admincount:False})
MATCH (m:User)
WHERE NOT m.name = n.name
MATCH p=allShortestPaths((n)-[r:AllExtendedRights|ForceChangePassword|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner*1..]->(m))
RETURN p
'''
).data()

[]

## Workstations a user can RDP into
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find workstations a user can RDP into.


In [29]:
graph.run(
'''
MATCH p=(g:Group)-[:CanRDP]->(c:Computer)
WHERE g.objectid ENDS WITH '-513' AND NOT c.operatingsystem CONTAINS 'Server'
RETURN p
'''
).data()

[]

## All Domain Users Edges Against all Computers
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find all the privileges (edges) of the domain users against the domain computers (e.g. CanRDP, AdminTo etc. HasSession edge is not included)


In [30]:
graph.run(
'''
MATCH p1=shortestPath(((u1:User)-[r1:MemberOf*1..]->(g1:Group)))
MATCH p2=(u1)-[*1]->(c:Computer)
RETURN p2
'''
).data()

[{'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 {'p2': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL)},
 

## All Logged In Admins
**Author:** Walter.Legowski (@SadProcessor)

**Description:** List of all logged in administrators


In [31]:
graph.run(
'''
MATCH 
p=(a:Computer)-[r:HasSession]->(b:User) 
WITH a,b,r 
MATCH 
p=shortestPath((b)-[:AdminTo|MemberOf*1..]->(a)) 
RETURN b,a,r
'''
).data()

[{'b': (_11:Base:User {admincount: true, description: 'Built-in account for administering the computer/domain', distinguishedname: 'CN=Administrator,CN=Users,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', dontreqpreauth: false, enabled: true, hasspn: false, highvalue: false, lastlogon: 1586793173.0, lastlogontimestamp: 1586738080.0, name: 'ADMINISTRATOR@CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-500', owned: false, passwordnotreqd: false, pwdlastset: 1560189819.0, pwdneverexpires: true, sensitive: false, serviceprincipalnames: [], sidhistory: [], unconstraineddelegation: false}),
  'a': (_40:Base:Computer {distinguishedname: 'CN=PCI-SERVER-001,OU=PCI Computers,OU=PCI Enclave,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', enabled: true, haslaps: false, highvalue: false, lastlogontimestamp: 1586551062.0, name: 'PCI-SERVER-001.CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-1617', operatingsystem: 'Windows Server 2016 Standard', owned: false, pwdla

## All Domain Admins
**Author:** Walter.Legowski (@SadProcessor)

**Description:** List of all domain admins


In [32]:
graph.run(
'''
MATCH (n:Group) WHERE n.name =~ "(?i).*DOMAIN ADMINS.*"
WITH n 
MATCH (n)<-[r:MemberOf*1..]-(m) 
RETURN n,r,m
'''
).data()

[{'n': (_20:Base:Group {admincount: true, description: 'Designated administrators of the domain', distinguishedname: 'CN=Domain Admins,CN=Users,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', highvalue: true, name: 'DOMAIN ADMINS@CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-512'}),
  'r': [(SQL01@CONTOSO.LOCAL)-[:MemberOf {isacl: false}]->(DOMAIN ADMINS@CONTOSO.LOCAL)],
  'm': (_38:Base:User {admincount: true, distinguishedname: 'CN=SQL01,OU=SQLUsers,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', dontreqpreauth: false, enabled: true, hasspn: false, highvalue: false, lastlogon: 1581142464.0, lastlogontimestamp: 1580846712.0, name: 'SQL01@CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-1606', owned: false, passwordnotreqd: false, pwdlastset: 1580169201.0, pwdneverexpires: false, sensitive: false, serviceprincipalnames: [], sidhistory: [], unconstraineddelegation: false})},
 {'n': (_20:Base:Group {admincount: true, description: 'Designated administrat

## Computers with Unconstrained Delegation
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find all computers with Unconstrained Delegation


In [33]:
graph.run(
'''
MATCH (c:Computer {unconstraineddelegation:true})
RETURN c
'''
).data()

[{'c': (_41:Base:Computer {distinguishedname: 'CN=DC-2016-001,OU=Domain Controllers,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', enabled: true, haslaps: false, highvalue: false, lastlogontimestamp: 1586550660.0, name: 'DC-2016-001.CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-1000', operatingsystem: 'Windows Server 2016 Standard', owned: false, pwdlastset: 1586379557.0, serviceprincipalnames: ['Dfsr-12F9A27C-BF97-4787-9364-D31B6C55EB04/DC-2016-001.contoso.local', 'ldap/DC-2016-001.contoso.local/ForestDnsZones.contoso.local', 'ldap/DC-2016-001.contoso.local/DomainDnsZones.contoso.local', 'DNS/DC-2016-001.contoso.local', 'GC/DC-2016-001.contoso.local/contoso.local', 'RestrictedKrbHost/DC-2016-001.contoso.local', 'RestrictedKrbHost/DC-2016-001', 'RPC/163efdab-a02f-4b72-8de1-b45b21b62341._msdcs.contoso.local', 'HOST/DC-2016-001/CONTOSO', 'HOST/DC-2016-001.contoso.local/CONTOSO', 'HOST/DC-2016-001', 'HOST/DC-2016-001.contoso.local', 'HOST/DC-2016-001.contoso.local/cont

## Unprivileged Users ACL abusing Computers
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find interesting edges related to “ACL Abuse” that unprivileged users have against computers


In [34]:
graph.run(
'''
MATCH (n:User {admincount:False})
MATCH p=allShortestPaths((n)-[r:AllExtendedRights|GenericAll|GenericWrite|Owns|WriteDacl|WriteOwner|AdminTo|CanRDP|ExecuteDCOM|ForceChangePassword*1..]->(m:Computer))
RETURN p
'''
).data()

[]

## User Sessions in a Specific Domain
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find all sessions any user in a specific domain has.


In [35]:
graph.run(
'''
MATCH p=(m:Computer)-[r:HasSession]->(n:User {domain: "CONTOSO.LOCAL"})
RETURN p
'''
).data()

[{'p': (PCI-SERVER-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ADMINISTRATOR@CONTOSO.LOCAL)},
 {'p': (RD-WIN10-001.CONTOSO.LOCAL)-[:HasSession {isacl: false}]->(ACHILES@CONTOSO.LOCAL)}]

## Top 10 Users with Most Local Admin Rights
**Author:** Walter.Legowski (@SadProcessor)

**Description:** List of top 10 users with most local admin rights


In [36]:
graph.run(
'''
MATCH (n:User),(m:Computer),(n)-[r:AdminTo]->(m)
WHERE NOT n.name STARTS WITH 'ANONYMOUS LOGON' AND NOT n.name='' WITH n, count(r) as rel_count
ORDER BY rel_count desc 
LIMIT 10 
MATCH (m)<-[r:AdminTo]-(n) 
RETURN n,r,m 
'''
).data()

[{'n': (_11:Base:User {admincount: true, description: 'Built-in account for administering the computer/domain', distinguishedname: 'CN=Administrator,CN=Users,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', dontreqpreauth: false, enabled: true, hasspn: false, highvalue: false, lastlogon: 1586793173.0, lastlogontimestamp: 1586738080.0, name: 'ADMINISTRATOR@CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-174133717-528793688-500', owned: false, passwordnotreqd: false, pwdlastset: 1560189819.0, pwdneverexpires: true, sensitive: false, serviceprincipalnames: [], sidhistory: [], unconstraineddelegation: false}),
  'r': (ADMINISTRATOR@CONTOSO.LOCAL)-[:AdminTo {fromgpo: false, isacl: false}]->(DC-2016-001.CONTOSO.LOCAL),
  'm': (_41:Base:Computer {distinguishedname: 'CN=DC-2016-001,OU=Domain Controllers,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', enabled: true, haslaps: false, highvalue: false, lastlogontimestamp: 1586550660.0, name: 'DC-2016-001.CONTOSO.LOCAL', objectid: 'S-1-5-21-153951712-17

## View all GPOs
**Author:** Ryan Hausknecht (@haus3c)

**Description:** View all GPOs


In [37]:
graph.run(
'''
MATCH (n:GPO)
RETURN n
'''
).data()

[{'n': (_9:Base:GPO {distinguishedname: 'CN={AC4ACFC1-0578-4251-9BBD-E1D0352A5CAF},CN=Policies,CN=System,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', gpcpath: '\\\\contoso.local\\SysVol\\contoso.local\\Policies\\{AC4ACFC1-0578-4251-9BBD-E1D0352A5CAF}', highvalue: false, name: 'PROTECT PCL ENCLAVE@CONTOSO.LOCAL', objectid: 'F9F3C61B-D37D-41EA-A064-71B26119E448'})},
 {'n': (_10:Base:GPO {distinguishedname: 'CN={6AC1786C-016F-11D2-945F-00C04fB984F9},CN=Policies,CN=System,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', gpcpath: '\\\\contoso.local\\sysvol\\contoso.local\\Policies\\{6AC1786C-016F-11D2-945F-00C04fB984F9}', highvalue: false, name: 'DEFAULT DOMAIN CONTROLLERS POLICY@CONTOSO.LOCAL', objectid: '6930F38E-81B5-497F-B18B-E770DF862D89'})},
 {'n': (_120:Base:GPO {distinguishedname: 'CN={31B2F340-016D-11D2-945F-00C04FB984F9},CN=Policies,CN=System,DC=contoso,DC=local', domain: 'CONTOSO.LOCAL', gpcpath: '\\\\contoso.local\\sysvol\\contoso.local\\Policies\\{31B2F340-016D-11D2-945F-00C04F

## SPNs with keywords
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find SPNs with keywords (swap SQL with whatever)


In [38]:
graph.run(
'''
MATCH (u:User)
WHERE ANY (x IN u.serviceprincipalnames WHERE toUpper(x) CONTAINS 'SQL')
RETURN u.name
'''
).data()

[{'u.name': 'SQL2014@CONTOSO.LOCAL'}]

## Group with keywords
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find a group with keywords. E.g. SQL ADMINS or SQL 2017 ADMINS


In [39]:
graph.run(
'''
MATCH (g:Group)
WHERE g.name =~ '(?i).SQL.ADMIN.*'
RETURN g
'''
).data()

[]

## Kerberoastable Users with passwords last set > 5 years
**Author:** Ryan Hausknecht (@haus3c)

**Description:** Find All Users with an SPN/Find all Kerberoastable Users with passwords last set > 5 years ago


In [40]:
graph.run(
'''
MATCH (u:User)
WHERE u.hasspn=true AND u.pwdlastset < (datetime().epochseconds - (1825 * 86400)) AND NOT u.pwdlastset IN [-1.0, 0.0]
RETURN u.name, u.pwdlastset order by u.pwdlastset
'''
).data()

[]

## DA sessions not on a certain group
**Author:** Ryan Hausknecht (@haus3c)

**Description:** DA sessions not on a certain group (e.g. domain controllers).


In [41]:
graph.run(
'''
OPTIONAL MATCH (c:Computer)-[:MemberOf]->(t:Group)
WHERE NOT t.name = 'DOMAIN CONTROLLERS@CONTOSO.LOCAL' WITH c as NonDC
MATCH p=(NonDC)-[:HasSession]->(n:User)-[:MemberOf]->(g:Group {name:'DOMAIN ADMINS@CONTOSO.LOCAL'})
RETURN DISTINCT (n.name) as Username, COUNT(DISTINCT(NonDC)) as Connexions
ORDER BY COUNT(DISTINCT(NonDC)) DESC   
'''
).data()

[{'Username': 'ADMINISTRATOR@CONTOSO.LOCAL', 'Connexions': 1}]